In [2]:
import pandas as pd
import keras
import sklearn
import numpy as np

# Este notebook es la fase final de enriquecimiento y preprocesado antes de entrenar

Carga de la tabla trabajada de transacciones

In [3]:
data_trx = pd.read_pickle('dt_trxpse_V4.pkl.gzip')

In [4]:
data_trx.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,primer_subsector,segundo_subsector,tercer_subsector,primer_descripcion,segundo_descripcion,tercer_descripcion,trx_mes_corrido
0,215120528,17734,277551.29,Pago OnLine Amway,CC,NaN,NaN,NaN,NaN,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,863895.90
1,215120525,248749,165555.19,Pago por iRecaudo de Claro Soluciones Móviles,NaN,NaN,NaN,NaN,NaN,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,691192.76
2,215120522,11856,32229.02,Pago de Saldo,NaN,NaN,MEDIOS DE COMUNICACION,TELEFONIA FIJA,Servicios de telefonía fija,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,32229.02
3,215120581,212722,381295.37,CC,NaN,NaN,NaN,NaN,NaN,2016-09-01 00:01:08,cc,vacio,vacio,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,488658.23
4,215120539,283039,321851.13,Vuelo MDE PSO,CC,NaN,NaN,NaN,NaN,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,NINGUNO,321851.13


In [5]:
data_trx['id_cliente'] = data_trx['id_cliente'].astype(int)

Carga de la tabla de pagadores

In [6]:
data_pagadores = pd.read_csv('Pagadores_v5_con_clusters.csv')

In [7]:
data_pagadores = data_pagadores.drop(['Unnamed: 0','Unnamed: 0.1','categoria_2_clusters','flag','edad_v1','edad'],axis = 1)

In [8]:
data_pagadores.head()

,id_cliente,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,ingreso_rango,ingreso_min,ingreso_max,edad_v2,categoria_4_clusters
0,18.0,PERSONAL PLUS,5,O,U,M,M,e. (4.4 5.5MM],4400001.01,5.500000e+06,4.449294,1
1,32.0,PERSONAL PLUS,E,F,T,M,M,i. (8.7 Inf),8700001.01,1.000000e+101,3.481206,1
2,41.0,EMPRENDEDOR,3,O,I,W,M,b. (1.1 2.2MM],1100001.01,2.200000e+06,4.287946,3
3,47.0,EMPRENDEDOR,7,I,I,I,M,c. (2.2 3.3MM],2200001.01,3.300000e+06,3.965250,3
4,71.0,PERSONAL,5,O,S,M,M,e. (4.4 5.5MM],4400001.01,5.500000e+06,3.400532,1


Carga de keywords de clusters

In [9]:
data_pagadores['id_cliente'] = data_pagadores['id_cliente'].astype(int)

In [10]:
data_pagadores.head()

,id_cliente,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,ingreso_rango,ingreso_min,ingreso_max,edad_v2,categoria_4_clusters
0,18,PERSONAL PLUS,5,O,U,M,M,e. (4.4 5.5MM],4400001.01,5.500000e+06,4.449294,1
1,32,PERSONAL PLUS,E,F,T,M,M,i. (8.7 Inf),8700001.01,1.000000e+101,3.481206,1
2,41,EMPRENDEDOR,3,O,I,W,M,b. (1.1 2.2MM],1100001.01,2.200000e+06,4.287946,3
3,47,EMPRENDEDOR,7,I,I,I,M,c. (2.2 3.3MM],2200001.01,3.300000e+06,3.965250,3
4,71,PERSONAL,5,O,S,M,M,e. (4.4 5.5MM],4400001.01,5.500000e+06,3.400532,1


In [11]:
data_pagadores.dtypes

id_cliente                int32
seg_str                  object
ocupacion                object
tipo_vivienda            object
nivel_academico          object
estado_civil             object
genero                   object
ingreso_rango            object
ingreso_min             float64
ingreso_max             float64
edad_v2                 float64
categoria_4_clusters      int64
dtype: object

In [12]:
keywords_clusters = pd.read_csv('cluster_keywords.csv')

In [13]:
keywords_clusters.columns = ['categoria_4_clusters','nombre','key1','key2','key3']

In [14]:
keywords_clusters.dtypes

categoria_4_clusters     int64
nombre                  object
key1                    object
key2                    object
key3                    object
dtype: object

In [15]:
data_trx_pagadores = data_trx.merge(data_pagadores, on = 'id_cliente', how='left').merge(keywords_clusters, on = 'categoria_4_clusters', how='left')

In [16]:
data_trx_pagadores.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable,...,genero,ingreso_rango,ingreso_min,ingreso_max,edad_v2,categoria_4_clusters,nombre,key1,key2,key3
0,215120528,17734,277551.29,Pago OnLine Amway,CC,NaN,NaN,NaN,NaN,2016-09-01 00:00:04,...,M,i. (8.7 Inf),8700001.01,1.000000e+101,0.980313,1,black,mayor,rico,profesional
1,215120525,248749,165555.19,Pago por iRecaudo de Claro Soluciones Móviles,NaN,NaN,NaN,NaN,NaN,2016-09-01 00:00:50,...,F,d. (3.3 4.4MM],3300001.01,4.400000e+06,-0.633167,0,universitario,joven,pobre,estudiante
2,215120522,11856,32229.02,Pago de Saldo,NaN,NaN,MEDIOS DE COMUNICACION,TELEFONIA FIJA,Servicios de telefonía fija,2016-09-01 00:00:59,...,M,b. (1.1 2.2MM],1100001.01,2.200000e+06,-0.229797,2,platino,adulto,acomodado,tecnico
3,215120581,212722,381295.37,CC,NaN,NaN,NaN,NaN,NaN,2016-09-01 00:01:08,...,M,c. (2.2 3.3MM],2200001.01,3.300000e+06,-0.794515,2,platino,adulto,acomodado,tecnico
4,215120539,283039,321851.13,Vuelo MDE PSO,CC,NaN,NaN,NaN,NaN,2016-09-01 00:01:38,...,M,b. (1.1 2.2MM],1100001.01,2.200000e+06,-0.633167,0,universitario,joven,pobre,estudiante


# Esta tabla es la consolidada de todos los enhancements de datos que hemos realizado.

# Procedemos a hacer un labeling y preparala para el BoW y asi correr el embedding.

In [17]:
label_data = pd.read_csv('mapeo_subsector_cat_v4.csv')

In [18]:
label_data.head()

,cat,subsector,descripcion
0,Agro,AGROQUIMICOS,Fabricación de plaguicidas y otros productos q...
1,Agro,DISTRIBUIDORES PRODUCTOS CONSUMO MASIVO,Comercio al por mayor de materias primas agrop...
2,Ahorro,OTROS SERVICIOS FINANCIEROS,Actividades de las cooperativas financieras
3,Ahorro,OTROS SERVICIOS FINANCIEROS,Actividades financieras de fondos de empleados...
4,Ahorro,PERSONAS,Asalariados


In [19]:
from sklearn import preprocessing as sklp 

In [20]:
data_label_v1 = data_trx_pagadores.merge(label_data, on=['subsector','descripcion'], how='left').drop(['sector','subsector','descripcion'],axis=1)

In [21]:
data_label_v1.columns

Index(['id_trn_ach', 'id_cliente', 'valor_trx', 'ref1', 'ref2', 'ref3',
       'fecha_usable', 'ref1_exp_nlp', 'ref2_exp_nlp', 'ref3_exp_nlp',
       'primer_subsector', 'segundo_subsector', 'tercer_subsector',
       'primer_descripcion', 'segundo_descripcion', 'tercer_descripcion',
       'trx_mes_corrido', 'seg_str', 'ocupacion', 'tipo_vivienda',
       'nivel_academico', 'estado_civil', 'genero', 'ingreso_rango',
       'ingreso_min', 'ingreso_max', 'edad_v2', 'categoria_4_clusters',
       'nombre', 'key1', 'key2', 'key3', 'cat'],
      dtype='object')

In [22]:
label_data.columns = ['cat','primer_subsector','primer_descripcion']

In [23]:
data_label_v2 = data_label_v1.merge(label_data, on=['primer_subsector','primer_descripcion'], how='left', suffixes=['_target','_past1']).drop(['primer_subsector','primer_descripcion'],axis=1)

In [24]:
data_label_v2.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,...,ingreso_min,ingreso_max,edad_v2,categoria_4_clusters,nombre,key1,key2,key3,cat_target,cat_past1
0,215120528,17734,277551.29,Pago OnLine Amway,CC,NaN,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,...,8700001.01,1.000000e+101,0.980313,1,black,mayor,rico,profesional,NaN,NaN
1,215120525,248749,165555.19,Pago por iRecaudo de Claro Soluciones Móviles,NaN,NaN,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,...,3300001.01,4.400000e+06,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN
2,215120522,11856,32229.02,Pago de Saldo,NaN,NaN,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,...,1100001.01,2.200000e+06,-0.229797,2,platino,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN
3,215120581,212722,381295.37,CC,NaN,NaN,2016-09-01 00:01:08,cc,vacio,vacio,...,2200001.01,3.300000e+06,-0.794515,2,platino,adulto,acomodado,tecnico,NaN,NaN
4,215120539,283039,321851.13,Vuelo MDE PSO,CC,NaN,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,...,1100001.01,2.200000e+06,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN


In [25]:
label_data.columns = ['cat','segundo_subsector','segundo_descripcion']
data_label_v3 = data_label_v2.merge(label_data, on=['segundo_subsector','segundo_descripcion'], how='left').drop(['segundo_subsector','segundo_descripcion'],axis=1)

In [26]:
data_label_v3.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,...,ingreso_max,edad_v2,categoria_4_clusters,nombre,key1,key2,key3,cat_target,cat_past1,cat
0,215120528,17734,277551.29,Pago OnLine Amway,CC,NaN,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,...,1.000000e+101,0.980313,1,black,mayor,rico,profesional,NaN,NaN,NaN
1,215120525,248749,165555.19,Pago por iRecaudo de Claro Soluciones Móviles,NaN,NaN,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,...,4.400000e+06,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN
2,215120522,11856,32229.02,Pago de Saldo,NaN,NaN,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,...,2.200000e+06,-0.229797,2,platino,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN
3,215120581,212722,381295.37,CC,NaN,NaN,2016-09-01 00:01:08,cc,vacio,vacio,...,3.300000e+06,-0.794515,2,platino,adulto,acomodado,tecnico,NaN,NaN,NaN
4,215120539,283039,321851.13,Vuelo MDE PSO,CC,NaN,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,...,2.200000e+06,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN


In [27]:
label_data.columns = ['cat','tercer_subsector','tercer_descripcion']
data_label_v4 = data_label_v3.merge(label_data, on=['tercer_subsector','tercer_descripcion'], how='left',suffixes=['_past2','_past3']).drop(['tercer_subsector','tercer_descripcion'],axis=1)

In [28]:
data_label_v4.head(1000000)

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,...,edad_v2,categoria_4_clusters,nombre,key1,key2,key3,cat_target,cat_past1,cat_past2,cat_past3
0,215120528,17734,277551.29,Pago OnLine Amway,CC,NaN,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,...,0.980313,1,black,mayor,rico,profesional,NaN,NaN,NaN,NaN
1,215120525,248749,165555.19,Pago por iRecaudo de Claro Soluciones Móviles,NaN,NaN,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,...,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN,NaN
2,215120522,11856,32229.02,Pago de Saldo,NaN,NaN,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,...,-0.229797,2,platino,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN,NaN
3,215120581,212722,381295.37,CC,NaN,NaN,2016-09-01 00:01:08,cc,vacio,vacio,...,-0.794515,2,platino,adulto,acomodado,tecnico,NaN,NaN,NaN,NaN
4,215120539,283039,321851.13,Vuelo MDE PSO,CC,NaN,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,...,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN,NaN
5,215120568,144266,154522.68,Colaboración para el proyecto Experiencia Pana...,CC,NaN,2016-09-01 00:01:54,"[colaboracion, proyecto, experiencia, panal, g...",cc,cc,...,-0.310471,2,platino,adulto,acomodado,tecnico,NaN,NaN,NaN,NaN
6,215120607,297198,229105.39,Pago OnLine Amway,CC,NaN,2016-09-01 00:02:13,"[pago, online, amway]",cc,cc,...,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN,NaN
7,215120583,134176,162307.40,Pago facturas: CMR,CC,NaN,2016-09-01 00:02:29,"[pago, facturas, cmr]",cc,cc,...,-0.149123,2,platino,adulto,acomodado,tecnico,NaN,NaN,NaN,NaN
8,215120619,135587,247878.32,CC,NaN,NaN,2016-09-01 00:02:38,cc,vacio,vacio,...,0.657617,1,black,mayor,rico,profesional,NaN,NaN,NaN,NaN
9,215120586,241516,26804.07,Pago de Saldo,NaN,NaN,2016-09-01 00:02:44,"[pago, saldo]",vacio,vacio,...,NaN,NaN,NaN,NaN,NaN,NaN,Tecnología y comunicaciones,NaN,NaN,NaN


In [29]:
data_label_v4.dtypes

id_trn_ach                      object
id_cliente                       int32
valor_trx                      float64
ref1                            object
ref2                            object
ref3                           float64
fecha_usable            datetime64[ns]
ref1_exp_nlp                    object
ref2_exp_nlp                    object
ref3_exp_nlp                    object
trx_mes_corrido                float64
seg_str                         object
ocupacion                       object
tipo_vivienda                   object
nivel_academico                 object
estado_civil                    object
genero                          object
ingreso_rango                   object
ingreso_min                    float64
ingreso_max                    float64
edad_v2                        float64
categoria_4_clusters            object
nombre                          object
key1                            object
key2                            object
key3                     

# Hasta aca, termina el transformation del label. Se considera de aca para arriba un universo donde solo esta el label de nosotros, la columna es cat_target

In [30]:
data_label_v5 = data_label_v4.drop(['ref1','ref2','ref3'],axis = 1)

In [31]:
data_label_v5.head()

,id_trn_ach,id_cliente,valor_trx,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,trx_mes_corrido,seg_str,ocupacion,...,edad_v2,categoria_4_clusters,nombre,key1,key2,key3,cat_target,cat_past1,cat_past2,cat_past3
0,215120528,17734,277551.29,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,863895.90,EMPRENDEDOR,8,...,0.980313,1,black,mayor,rico,profesional,NaN,NaN,NaN,NaN
1,215120525,248749,165555.19,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,691192.76,PERSONAL,1,...,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN,NaN
2,215120522,11856,32229.02,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,32229.02,PERSONAL,1,...,-0.229797,2,platino,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN,NaN
3,215120581,212722,381295.37,2016-09-01 00:01:08,cc,vacio,vacio,488658.23,PERSONAL,P,...,-0.794515,2,platino,adulto,acomodado,tecnico,NaN,NaN,NaN,NaN
4,215120539,283039,321851.13,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,321851.13,PERSONAL,2,...,-0.633167,0,universitario,joven,pobre,estudiante,NaN,NaN,NaN,NaN


In [32]:
def function_aux_nlp(x):
    if isinstance(x, (list,)):
        return ','.join(map(str, x))
    else:
        return x

In [33]:
data_label_v5['ref1_exp_nlp_aux'] = data_label_v5['ref1_exp_nlp'].apply(lambda x:  function_aux_nlp(x))
data_label_v5['ref2_exp_nlp_aux'] = data_label_v5['ref2_exp_nlp'].apply(lambda x:  function_aux_nlp(x))
data_label_v5['ref3_exp_nlp_aux'] = data_label_v5['ref3_exp_nlp'].apply(lambda x:  function_aux_nlp(x))

In [34]:
data_label_v5.head()

,id_trn_ach,id_cliente,valor_trx,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,trx_mes_corrido,seg_str,ocupacion,...,key1,key2,key3,cat_target,cat_past1,cat_past2,cat_past3,ref1_exp_nlp_aux,ref2_exp_nlp_aux,ref3_exp_nlp_aux
0,215120528,17734,277551.29,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,863895.90,EMPRENDEDOR,8,...,mayor,rico,profesional,NaN,NaN,NaN,NaN,"pago,online,amway",cc,cc
1,215120525,248749,165555.19,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,691192.76,PERSONAL,1,...,joven,pobre,estudiante,NaN,NaN,NaN,NaN,"pago,irecaudo,soluciones,moviles",vacio,vacio
2,215120522,11856,32229.02,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,32229.02,PERSONAL,1,...,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo",vacio,vacio
3,215120581,212722,381295.37,2016-09-01 00:01:08,cc,vacio,vacio,488658.23,PERSONAL,P,...,adulto,acomodado,tecnico,NaN,NaN,NaN,NaN,cc,vacio,vacio
4,215120539,283039,321851.13,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,321851.13,PERSONAL,2,...,joven,pobre,estudiante,NaN,NaN,NaN,NaN,"vuelo,mde,pso",cc,cc


In [35]:
data_label_v5['final_nlp'] = data_label_v5[['ref1_exp_nlp_aux', 'ref2_exp_nlp_aux','ref3_exp_nlp_aux']].apply(lambda x: ','.join(x), axis=1)

In [36]:
data_label_v5.head()

,id_trn_ach,id_cliente,valor_trx,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp,trx_mes_corrido,seg_str,ocupacion,...,key2,key3,cat_target,cat_past1,cat_past2,cat_past3,ref1_exp_nlp_aux,ref2_exp_nlp_aux,ref3_exp_nlp_aux,final_nlp
0,215120528,17734,277551.29,2016-09-01 00:00:04,"[pago, online, amway]",cc,cc,863895.90,EMPRENDEDOR,8,...,rico,profesional,NaN,NaN,NaN,NaN,"pago,online,amway",cc,cc,"pago,online,amway,cc,cc"
1,215120525,248749,165555.19,2016-09-01 00:00:50,"[pago, irecaudo, soluciones, moviles]",vacio,vacio,691192.76,PERSONAL,1,...,pobre,estudiante,NaN,NaN,NaN,NaN,"pago,irecaudo,soluciones,moviles",vacio,vacio,"pago,irecaudo,soluciones,moviles,vacio,vacio"
2,215120522,11856,32229.02,2016-09-01 00:00:59,"[pago, saldo]",vacio,vacio,32229.02,PERSONAL,1,...,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo",vacio,vacio,"pago,saldo,vacio,vacio"
3,215120581,212722,381295.37,2016-09-01 00:01:08,cc,vacio,vacio,488658.23,PERSONAL,P,...,acomodado,tecnico,NaN,NaN,NaN,NaN,cc,vacio,vacio,"cc,vacio,vacio"
4,215120539,283039,321851.13,2016-09-01 00:01:38,"[vuelo, mde, pso]",cc,cc,321851.13,PERSONAL,2,...,pobre,estudiante,NaN,NaN,NaN,NaN,"vuelo,mde,pso",cc,cc,"vuelo,mde,pso,cc,cc"


In [37]:
data_label_v5.dtypes

id_trn_ach                      object
id_cliente                       int32
valor_trx                      float64
fecha_usable            datetime64[ns]
ref1_exp_nlp                    object
ref2_exp_nlp                    object
ref3_exp_nlp                    object
trx_mes_corrido                float64
seg_str                         object
ocupacion                       object
tipo_vivienda                   object
nivel_academico                 object
estado_civil                    object
genero                          object
ingreso_rango                   object
ingreso_min                    float64
ingreso_max                    float64
edad_v2                        float64
categoria_4_clusters            object
nombre                          object
key1                            object
key2                            object
key3                            object
cat_target                      object
cat_past1                       object
cat_past2                

In [38]:
data_label_v5.columns

Index(['id_trn_ach', 'id_cliente', 'valor_trx', 'fecha_usable', 'ref1_exp_nlp',
       'ref2_exp_nlp', 'ref3_exp_nlp', 'trx_mes_corrido', 'seg_str',
       'ocupacion', 'tipo_vivienda', 'nivel_academico', 'estado_civil',
       'genero', 'ingreso_rango', 'ingreso_min', 'ingreso_max', 'edad_v2',
       'categoria_4_clusters', 'nombre', 'key1', 'key2', 'key3', 'cat_target',
       'cat_past1', 'cat_past2', 'cat_past3', 'ref1_exp_nlp_aux',
       'ref2_exp_nlp_aux', 'ref3_exp_nlp_aux', 'final_nlp'],
      dtype='object')

In [39]:
data_label_v6 = data_label_v5.drop(['ref1_exp_nlp',
       'ref2_exp_nlp', 'ref3_exp_nlp','ref1_exp_nlp_aux',
       'ref2_exp_nlp_aux', 'ref3_exp_nlp_aux'], axis = 1)

In [40]:
data_label_v6.dtypes

id_trn_ach                      object
id_cliente                       int32
valor_trx                      float64
fecha_usable            datetime64[ns]
trx_mes_corrido                float64
seg_str                         object
ocupacion                       object
tipo_vivienda                   object
nivel_academico                 object
estado_civil                    object
genero                          object
ingreso_rango                   object
ingreso_min                    float64
ingreso_max                    float64
edad_v2                        float64
categoria_4_clusters            object
nombre                          object
key1                            object
key2                            object
key3                            object
cat_target                      object
cat_past1                       object
cat_past2                       object
cat_past3                       object
final_nlp                       object
dtype: object

In [41]:
data_label_v6.final_nlp.head(20)

0                               pago,online,amway,cc,cc
1          pago,irecaudo,soluciones,moviles,vacio,vacio
2                                pago,saldo,vacio,vacio
3                                        cc,vacio,vacio
4                                   vuelo,mde,pso,cc,cc
5     colaboracion,proyecto,experiencia,panal,girard...
6                               pago,online,amway,cc,cc
7                               pago,facturas,cmr,cc,cc
8                                        cc,vacio,vacio
9                                pago,saldo,vacio,vacio
10         pago,irecaudo,soluciones,moviles,vacio,vacio
11                              pago,online,amway,cc,cc
12                              pago,facturas,cmr,cc,cc
13                 empresas,publicas,medellin,esp,cc,cc
14                               pago,saldo,vacio,vacio
15                                       cc,vacio,vacio
16                            compra,falabellacom,cc,cc
17                    pago,reserva,txxvjz,bog,ey

Desde aca pasamos a un problema supervisado

In [42]:
corpus = []
for row in data_label_v6.final_nlp:
    a = row.split(',')
    for i in a:
        corpus.append(i)

In [43]:
corpus = list(set(corpus))

In [44]:
len(corpus)

398511

In [45]:
supervised = data_label_v6.cat_target.isnull()

In [46]:
data_supervised = data_label_v6[~supervised]

In [47]:
data_supervised.shape

(3285321, 25)

In [48]:
data_supervised.reset_index(inplace=True)

In [49]:
corpus_sup = []
for row in data_supervised.final_nlp:
    q = row.split(',')
    for m in q:
        corpus_sup.append(m)

In [ ]:
data_supervised.to_pickle('data_supervised.pkl.gzip')

# Prepara el texto

In [160]:
from numpy import array
from keras.preprocessing.text import one_hot
docs = data_supervised['final_nlp']
# integer encode the documents
vocab_size = 10000
encoded_docs = [one_hot(d, vocab_size) for d in docs]


In [161]:
data_supervised['encoded_nlp'] = encoded_docs

D:\Anaconda\envs\dataton2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [162]:
data_supervised.columns

Index(['index', 'id_trn_ach', 'id_cliente', 'valor_trx', 'fecha_usable',
       'trx_mes_corrido', 'seg_str', 'ocupacion', 'tipo_vivienda',
       'nivel_academico', 'estado_civil', 'genero', 'ingreso_rango',
       'ingreso_min', 'ingreso_max', 'edad_v2', 'categoria_4_clusters',
       'nombre', 'key1', 'key2', 'key3', 'cat_target', 'cat_past1',
       'cat_past2', 'cat_past3', 'final_nlp', 'encoded_nlp'],
      dtype='object')

In [163]:
data_supervised['encoded_nlp']

0                                   [4395, 1362, 6057, 6057]
1                                   [4395, 1362, 6057, 6057]
2                       [2249, 9704, 1222, 7745, 3410, 3410]
3                                   [4395, 1362, 6057, 6057]
4                               [4395, 1896, 9795, 452, 452]
5                             [2931, 4395, 5442, 3410, 3410]
6                             [2931, 4395, 5442, 3410, 3410]
7                                   [4395, 1362, 6057, 6057]
8                                   [4395, 1362, 6057, 6057]
9                                   [4395, 1362, 6057, 6057]
10                                  [4395, 1362, 6057, 6057]
11                            [2931, 4395, 5442, 3410, 3410]
12                      [2249, 9704, 1222, 7745, 3410, 3410]
13                [5396, 4901, 2652, 7567, 5997, 5892, 5892]
14                            [2931, 4395, 5442, 3410, 3410]
15                            [2931, 4395, 5442, 3410, 3410]
16                      

In [1]:
# Truncate and pad the review sequences 
from keras.preprocessing import sequence 
max_texto_pse = 20 
data_supervised['encoded_nlp'] = sequence.pad_sequences(data_supervised['encoded_nlp'], maxlen=max_texto_pse) 

Using TensorFlow backend.


NameError: name 'data_supervised' is not defined

In [165]:
from sklearn import preprocessing

In [171]:
data_supervised = data_supervised.join(pd.get_dummies(data_supervised[['seg_str', 'ocupacion', 'tipo_vivienda',
       'nivel_academico', 'estado_civil', 'genero', 'ingreso_rango', 'categoria_4_clusters',
       'nombre', 'key1', 'key2', 'key3', 'cat_past1',
       'cat_past2', 'cat_past3']]), how='left')

MemoryError: 

In [170]:
data_supervised.head()

,index,id_trn_ach,id_cliente,valor_trx,fecha_usable,trx_mes_corrido,seg_str,ocupacion,tipo_vivienda,nivel_academico,...,nombre,key1,key2,key3,cat_target,cat_past1,cat_past2,cat_past3,final_nlp,encoded_nlp
0,2,215120522,11856,32229.02,2016-09-01 00:00:59,32229.02,PERSONAL,1,O,T,...,platino,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo,vacio,vacio","[4395, 1362, 6057, 6057]"
1,9,215120586,241516,26804.07,2016-09-01 00:02:44,26804.07,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo,vacio,vacio","[4395, 1362, 6057, 6057]"
2,13,215120682,206803,270247.63,2016-09-01 00:04:45,270247.63,PERSONAL,1,F,E,...,platino,adulto,acomodado,tecnico,Hogar,NaN,NaN,NaN,"empresas,publicas,medellin,esp,cc,cc","[2249, 9704, 1222, 7745, 3410, 3410]"
3,14,215120662,44531,78317.39,2016-09-01 00:04:50,78317.39,PERSONAL,1,I,U,...,universitario,joven,pobre,estudiante,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo,vacio,vacio","[4395, 1362, 6057, 6057]"
4,18,215120740,135203,1063830.22,2016-09-01 00:07:41,1642590.43,PERSONAL PLUS,1,F,U,...,black,mayor,rico,profesional,Servicios financieros,NaN,NaN,NaN,"pago,factura,asociado,bancoomevapfa,bancoomevapfa","[4395, 1896, 9795, 452, 452]"


In [155]:
#Preprocess for training
num_classes = 27
y_train = keras.utils.to_categorical(data_supervised['cat_target'], num_classes)

ValueError: invalid literal for int() with base 10: 'Tecnología y comunicaciones'